In [41]:
conn = sqlite3.connect('C:/Users/huber/OneDrive/NHL/DbMatchs.db')

#fonction qui crée une df du game log d'un gardien
def logGardien(idNHL):
    jsonJoueur = requests.get("https://statsapi.web.nhl.com/api/v1/people/" + idNHL + "/stats?stats=gameLog&season=20202021").json()
    df = pd.json_normalize(jsonJoueur["stats"][0]["splits"])
    gardienLogs = df[["season", "date", "isWin", "stat.shutouts"]]
    gardienLogs["gardien"] = idNHL
    return gardienLogs

#générer la liste de tous les gardiens de la table JOUEURS 
dfGardiens = pd.read_sql("SELECT idNHL FROM JOUEURS WHERE position == 'G'", conn)
gardiens = dfGardiens["idNHL"].apply(str)
listGardiens = gardiens.values.tolist()

#créer une df vide pour plus tard append le return de logGardien(idNHL) 
vraisCol = ["season", "date", "isWin", "stat.shutouts", "gardien"]
gameLogsG = pd.DataFrame(columns=vraisCol)
gameLogsG = gameLogsG.fillna(0)

#itérer logGardien(idNHL) à la liste des gardiens de la table JOUEURS et sauvegarder les résultats à gameLogsG
for i in listGardiens:
    try:
        gardienLogs = logGardien(i)
    except:
        print("fausse alarme (espoir) " + i)
    gameLogsG = gameLogsG.append(gardienLogs)
gameLogsG = gameLogsG.rename(columns = {"date" : "dateMatch"})
gameLogsG["dateMatch"] = pd.to_datetime(gameLogsG["dateMatch"])
gameLogsG

fausse alarme (espoir) 8482137
fausse alarme (espoir) 8481519


,season,dateMatch,isWin,stat.shutouts,gardien
0,20202021,2021-02-08,True,0,8475883
1,20202021,2021-02-06,True,0,8475883
2,20202021,2021-02-04,True,0,8475883
3,20202021,2021-01-30,False,0,8475883
4,20202021,2021-01-28,True,0,8475883
5,20202021,2021-01-26,True,0,8475883
6,20202021,2021-01-22,True,0,8475883
7,20202021,2021-01-20,False,0,8475883
8,20202021,2021-01-18,True,0,8475883
9,20202021,2021-01-15,False,0,8475883


In [53]:
# Faire le calcul de points avant de le mettre en SQL
a = gameLogsG["isWin"]
gameLogsG["esWin"] = a * 2
gameLogsG ["scorePool"] = gameLogsG["esWin"] + gameLogsG["stat.shutouts"]

# save la df des game logs en une table SQL
gameLogsG.to_sql("GARDIENS", conn, if_exists="replace", index=False)

# faire le update SQL